# MD Simulation Free-State Analysis

Caricare le dipendenze e pacchetti necessari per l'analisi

### 🔍 Caricamento delle dipendenze necessarie
Assicurarsi di aver scaricato tutte le dipendenze necessarie. Non andare avanti se dopo aver eseguito questa prima parte escono degli errori

In [5]:
# ✅ BLOCCO 1: Caricamento dipendenze
import os
import MDAnalysis as mda
from IPython.display import display, clear_output
import ipywidgets as widgets
from tqdm.notebook import tqdm

print("✅ Dipendenze caricate correttamente")


✅ Dipendenze caricate correttamente


### 🔄 Convertitore di file Multimodel.pdb -> trajectory.xtc
Questa cella esegue una conversione dei file multimodello generati da MacroModel in file di traiettoria leggibili da diversi programmi di analisi di dinamica molecolare.

**Ottenere i file multimodello (MULTIMODEL)**
- Dopo aver ottenuto i file *structure.maegz* di dinamica da Macromodel o DESMOND
- Aprire il promp dei comandi di Shodinger Maestro ( Shrodinger command Prompt) ed eseguite il *structconvert.exe*

In [2]:
# ✅ BLOCCO 1: Caricamento dipendenze
import os
import MDAnalysis as mda
from IPython.display import display, clear_output
import ipywidgets as widgets
from tqdm.notebook import tqdm
import time

print("✅ Dipendenze caricate correttamente")

# ✅ BLOCCO 2: Configurazione widget
path_input = widgets.Text(
    placeholder='Incolla il percorso completo del file PDB',
    description='Percorso:',
    layout={'width': '600px'}
)

load_button = widgets.Button(description="📂 Carica Struttura")
convert_button = widgets.Button(description="🔄 Converti in XTC")
progress = widgets.FloatProgress(value=0, min=0, max=1, description='Progresso:', bar_style='')
output = widgets.Output()

display(widgets.VBox([
    path_input,
    widgets.HBox([load_button, convert_button]),
    progress,
    output
]))

# ✅ BLOCCO 3: Funzioni core ottimizzate
def load_pdb_from_path(pdb_path):
    """Carica un PDB multi-model con gestione errori avanzata"""
    try:
        with output:
            clear_output()
            print("⏳ Caricamento in corso...")
        
        # Forza l'aggiornamento dell'output prima del caricamento
        time.sleep(0.1)
        
        if not os.path.exists(pdb_path):
            with output:
                clear_output()
                print(f"❌ File non trovato: {pdb_path}")
            return None
        
        u = mda.Universe(pdb_path)
        
        with output:
            clear_output()
            print(f"✅ Caricamento completato")
            print(f"Percorso: {os.path.abspath(pdb_path)}")
            print(f"Frame: {len(u.trajectory)}")
            print(f"Atomi: {len(u.atoms)}")
            print(f"Residui: {len(u.residues)}")
        
        return u
    
    except Exception as e:
        with output:
            clear_output()
            print(f"❌ Errore durante il caricamento")
            print(f"Tipo errore: {type(e).__name__}")
            print(f"Dettagli: {str(e)}")
        return None

def convert_to_xtc(universe):
    """Conversione in XTC con output ottimizzato"""
    try:
        output_path = os.path.join(os.path.dirname(path_input.value), "trajectory.xtc")
        n_frames = len(universe.trajectory)
        
        with output:
            clear_output()
            print("⏳ Conversione in corso...")
            print(f"Frame totali da processare: {n_frames}")
            print(f"File di output: {output_path}")
        
        progress.max = n_frames
        progress.value = 0
        
        # Conversione con gestione del rate limit
        update_interval = max(1, n_frames // 100)  # Aggiorna progresso ogni 1% o ogni frame
        with mda.Writer(output_path, universe.atoms.n_atoms) as W:
            for i, ts in enumerate(universe.trajectory):
                W.write(universe.atoms)
                if i % update_interval == 0:
                    progress.value = i
                    time.sleep(0.01)  # Riduce il rate dei messaggi
        
        progress.value = n_frames
        progress.bar_style = 'success'
        
        with output:
            clear_output()
            print("✅ Conversione completata con successo")
            print(f"File creato: {os.path.abspath(output_path)}")
            print(f"Dimensioni: {os.path.getsize(output_path)/1024/1024:.2f} MB")
        
        return True
    
    except Exception as e:
        with output:
            clear_output()
            print("❌ Errore durante la conversione")
            print(f"Tipo errore: {type(e).__name__}")
            print(f"Dettagli: {str(e)}")
        return False

# ✅ BLOCCO 4: Gestione eventi ottimizzata
def on_load_click(b):
    global u
    load_button.disabled = True
    try:
        u = load_pdb_from_path(path_input.value.strip())
        convert_button.disabled = not bool(u)
    finally:
        load_button.disabled = False

def on_convert_click(b):
    convert_button.disabled = True
    try:
        if 'u' not in globals() or not u:
            with output:
                clear_output()
                print("❌ Nessuna struttura caricata")
            return
        
        convert_to_xtc(u)
    finally:
        convert_button.disabled = False

load_button.on_click(on_load_click)
convert_button.on_click(on_convert_click)
convert_button.disabled = True

✅ Dipendenze caricate correttamente


### 📥 Analisi del Root Mean Square Deviation (RMSD)
In questa sezione, viene caricata la traiettoria della simulazione molecolare utilizzando MDTraj.

**Input richiesti:**
- File `.xtc` contenente la traiettoria.
- File `.pdb` o `.mae` contenente il file di topologia iniziale (struttura ligando).
Assicurati che i percorsi siano corretti e che i file siano nella stessa directory o accessibili dal percorso specificato.

In [11]:
import mdtraj as md
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import widgets, Layout, Output, VBox, HBox
from IPython.display import display, clear_output
import io
import base64
from matplotlib.backends.backend_pdf import PdfPages

# Widget per inserire path file PDB e XTC
pdb_path_widget = widgets.Text(
    value='',
    placeholder='Inserisci il path completo del file PDB',
    description='File PDB:',
    layout=Layout(width='70%')
)
xtc_path_widget = widgets.Text(
    value='',
    placeholder='Inserisci il path completo del file XTC',
    description='File XTC:',
    layout=Layout(width='70%')
)

# Widget per configurazione tempo
time_step_widget = widgets.FloatText(
    value=1.0,
    description='Time step:',
    step=0.1,
    layout=Layout(width='30%')
)

time_unit_widget = widgets.Dropdown(
    options=['ps', 'ns'],
    value='ps',
    description='Unità:',
    layout=Layout(width='30%')
)

time_config_box = HBox([time_step_widget, time_unit_widget])

# Widget per esportazione grafico
format_widget = widgets.Dropdown(
    options=[('PNG', 'png'), ('PDF', 'pdf'), ('SVG', 'svg')],
    value='png',
    description='Formato:',
    layout=Layout(width='30%')
)

download_button = widgets.Button(
    description='Scarica grafico',
    disabled=True,
    button_style='success',
    icon='download'
)

# Output widget per messaggi e grafico
output = Output()
plot_output = Output()

# Variabile globale per il grafico corrente
current_fig = None

def analyze_rmsd(b):
    global current_fig
    with output:
        clear_output()
        pdb_path = pdb_path_widget.value.strip()
        xtc_path = xtc_path_widget.value.strip()
        dt = time_step_widget.value
        time_unit = time_unit_widget.value
        
        # Controllo input
        if not pdb_path or not xtc_path:
            print("Errore: inserisci entrambi i path per PDB e XTC.")
            return
        
        if dt <= 0:
            print("Errore: il time step deve essere maggiore di 0")
            return
        
        try:
            traj = md.load(xtc_path, top=pdb_path)
        except Exception as e:
            print(f"Errore nel caricamento dei file: {e}")
            return
        
        # Calcolo RMSD rispetto al primo frame
        rmsd = md.rmsd(traj, traj, 0)
        
        # Conversione unità temporali
        if time_unit == 'ns':
            dt_ps = dt * 1000  # Converti ns in ps
            xlabel = 'Time (ns)'
            time_axis = np.arange(traj.n_frames) * dt  # Tempo in ns
        else:
            dt_ps = dt  # Mantieni in ps
            xlabel = 'Time (ps)'
            time_axis = np.arange(traj.n_frames) * dt  # Tempo in ps
        
        # Imposta range asse y (RMSD in Angstrom)
        rmsd_angstrom = rmsd * 10  # Converti nm in Å
        ymin = np.min(rmsd_angstrom)
        ymax = np.max(rmsd_angstrom)
        yrange = ymax - ymin
        if yrange == 0:
            ymin -= 0.1
            ymax += 0.1
        else:
            margin = yrange * 0.05
            ymin -= margin
            ymax += margin
        
        # Range asse x (tempo)
        xmin = 0
        xmax = time_axis[-1] if len(time_axis) > 0 else 1
        if xmin == xmax:
            xmax = xmin + 1
        
        # Plot
        with plot_output:
            clear_output()
            plt.close('all')  # Chiude eventuali figure precedenti
            fig, ax = plt.subplots(figsize=(8,5))
            ax.plot(time_axis, rmsd_angstrom, label='RMSD')
            ax.set_xlabel(xlabel)
            ax.set_ylabel('RMSD (Å)')
            ax.set_title('Analisi RMSD della traiettoria')
            ax.grid(True)
            ax.set_xlim(xmin, xmax)
            ax.set_ylim(ymin, ymax)
            ax.legend()
            
            current_fig = fig  # Salva la figura corrente
            plt.show()
            
        download_button.disabled = False
        print("Analisi completata con successo!")

def download_plot(b):
    if current_fig is None:
        with output:
            print("Nessun grafico disponibile per il download")
        return
    
    format = format_widget.value
    filename = f"RMSD_plot.{format}"
    
    with output:
        clear_output()
        try:
            if format == 'pdf':
                with PdfPages(filename) as pdf:
                    pdf.savefig(current_fig)
            else:
                current_fig.savefig(filename, format=format)
            
            print(f"Grafico esportato con successo come {filename}")
            
            # Crea un link per il download
            with open(filename, "rb") as f:
                data = f.read()
            b64 = base64.b64encode(data).decode()
            display(widgets.HTML(
                f'<a href="data:file/{format};base64,{b64}" download="{filename}">'
                f'Clicca qui per scaricare {filename}</a>'
            ))
            
        except Exception as e:
            print(f"Errore durante l'esportazione: {e}")

# Bottone per lanciare l'analisi
button = widgets.Button(description="Calcola RMSD")
button.on_click(analyze_rmsd)
download_button.on_click(download_plot)

# Mostra widget e output
display(VBox([
    pdb_path_widget, 
    xtc_path_widget, 
    widgets.Label("Configurazione asse temporale:"),
    time_config_box,
    button,
    widgets.Label("Esportazione grafico:"),
    HBox([format_widget, download_button]),
    output,
    plot_output
]))

### 🧪 Campionamento degli angoli glicosidici
In questa sezione, dev'essere caricata la traiettoria (file.xtc) della simulazione molecolare utilizzando MDTraj.
**Input richiesti:**
- File `.xtc` contenente la traiettoria.
- File `.pdb` o `.mae` contenente la topologia (struttura ligando).
Assicurati che i percorsi siano corretti e che i file siano nella stessa directory o accessibili dal percorso specificato.
Alla fine del processo, puoi scaricare i grafici in formato .png .pdf o .svg

In [12]:
import mdtraj as md
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import widgets, VBox, HBox, Output, Layout
from IPython.display import display, clear_output

# --- Caricamento file PDB e XTC tramite input path ---
pdb_path_widget = widgets.Text(description="Path PDB:", layout=Layout(width='70%'))
xtc_path_widget = widgets.Text(description="Path XTC:", layout=Layout(width='70%'))
load_button = widgets.Button(description="Carica Traiettoria")

output_load = Output()

def carica_traiettoria(b):
    with output_load:
        clear_output()
        pdb_path = pdb_path_widget.value.strip()
        xtc_path = xtc_path_widget.value.strip()
        try:
            global traj
            traj = md.load_xtc(xtc_path, top=pdb_path)
            print(f"Traiettoria caricata con {traj.n_frames} frame e {traj.n_atoms} atomi.")
            global residues_list
            residues_list = [f"{res.resSeq} {res.name}" for res in traj.topology.residues]
            print(f"Residui trovati: {len(residues_list)}")
            num_torsions_widget.disabled = False
        except Exception as e:
            print(f"Errore nel caricamento: {e}")

load_button.on_click(carica_traiettoria)

display(VBox([pdb_path_widget, xtc_path_widget, load_button, output_load]))

# --- Selezione numero torsioni ---
num_torsions_widget = widgets.BoundedIntText(
    value=1, min=1, max=10, step=1,
    description='N° torsioni:',
    disabled=True
)

output_torsions = Output()

def aggiorna_atomi_residuo(change, atoms_dd_list):
    val = change['new']
    if not val:
        for dd in atoms_dd_list:
            dd.options = []
            dd.value = None
        return
    resnum, resname = val.split()
    resnum = int(resnum)
    residue = None
    for r in traj.topology.residues:
        if r.resSeq == resnum and r.name == resname:
            residue = r
            break
    if residue is None:
        for dd in atoms_dd_list:
            dd.options = []
            dd.value = None
        return
    opts = [f"{atom.index} - {atom.name}" for atom in residue.atoms]
    for dd in atoms_dd_list:
        dd.options = opts
        dd.value = None

def estrai_indice_atom(atom_str):
    if not atom_str:
        raise ValueError("Atomo non selezionato")
    return int(atom_str.split(' - ')[0])

def crea_blocco_angolo(torsion_idx, angolo_tipo):
    """
    torsion_idx: int, indice torsione (es 0,1,...)
    angolo_tipo: str 'PHI' o 'PSI'
    """
    title = widgets.HTML(value=f"<b>Torsione #{torsion_idx+1} - Angolo {angolo_tipo}</b>")
    resA_dd = widgets.Dropdown(options=residues_list, description='Residuo A:', layout=Layout(width='45%'))
    resB_dd = widgets.Dropdown(options=residues_list, description='Residuo B:', layout=Layout(width='45%'))
    
    # PHI ha 2 atomi per residuo (4 totali)
    # PSI ha 1 atomo residuo A e 3 atomi residuo B (4 totali)
    if angolo_tipo == 'PHI':
        atoms_resA_dd = [widgets.Dropdown(description=f"Atomo A{i+1}:", layout=Layout(width='40%')) for i in range(2)]
        atoms_resB_dd = [widgets.Dropdown(description=f"Atomo B{i+1}:", layout=Layout(width='40%')) for i in range(2)]
    else:  # PSI
        atoms_resA_dd = [widgets.Dropdown(description="Atomo A1:", layout=Layout(width='40%'))]
        atoms_resB_dd = [widgets.Dropdown(description=f"Atomo B{i+1}:", layout=Layout(width='40%')) for i in range(3)]
    
    resA_dd.observe(lambda change: aggiorna_atomi_residuo(change, atoms_resA_dd), names='value')
    resB_dd.observe(lambda change: aggiorna_atomi_residuo(change, atoms_resB_dd), names='value')
    
    box_residui = HBox([resA_dd, resB_dd])
    box_atomsA = HBox(atoms_resA_dd)
    box_atomsB = HBox(atoms_resB_dd)
    
    blocco = VBox([title, box_residui, box_atomsA, box_atomsB])
    return blocco, atoms_resA_dd, atoms_resB_dd

def genera_widgets_torsioni(change):
    with output_torsions:
        clear_output()
        n = change['new']
        if n < 1:
            print("Inserisci un numero valido > 0")
            return
        
        torsion_blocks = []
        
        for i in range(n):
            phi_block, phi_atomsA, phi_atomsB = crea_blocco_angolo(i, 'PHI')
            psi_block, psi_atomsA, psi_atomsB = crea_blocco_angolo(i, 'PSI')
            
            output_plot = Output()
            fig_container = {"fig": None}
            
            # Widget per nome file e formato esportazione
            filename_widget = widgets.Text(
                value=f'torsione_{i+1}',
                description='Nome file:',
                layout=Layout(width='50%')
            )
            format_widget = widgets.Dropdown(
                options=['png', 'svg', 'pdf'],
                value='png',
                description='Formato:'
            )
            
            def calcola_e_plotta(b, i=i,
                               phi_atomsA=phi_atomsA, phi_atomsB=phi_atomsB,
                               psi_atomsA=psi_atomsA, psi_atomsB=psi_atomsB,
                               out=output_plot, fig_cont=fig_container):
                with out:
                    clear_output()
                    try:
                        # PHI = [A1, A2, B1, B2]
                        A1 = estrai_indice_atom(phi_atomsA[0].value)
                        A2 = estrai_indice_atom(phi_atomsA[1].value)
                        B1 = estrai_indice_atom(phi_atomsB[0].value)
                        B2 = estrai_indice_atom(phi_atomsB[1].value)
                        phi_angles = md.compute_dihedrals(traj, [[A1, A2, B1, B2]])
                        
                        # PSI = [A1, B1, B2, B3]
                        A1_psi = estrai_indice_atom(psi_atomsA[0].value)
                        B1_psi = estrai_indice_atom(psi_atomsB[0].value)
                        B2_psi = estrai_indice_atom(psi_atomsB[1].value)
                        B3_psi = estrai_indice_atom(psi_atomsB[2].value)
                        psi_angles = md.compute_dihedrals(traj, [[A1_psi, B1_psi, B2_psi, B3_psi]])
                        
                        phi_deg = np.degrees(phi_angles).flatten()
                        psi_deg = np.degrees(psi_angles).flatten()
                        
                        fig, ax = plt.subplots(figsize=(6,5))
                        scatter = ax.scatter(phi_deg, psi_deg, c=range(len(phi_deg)), cmap='viridis', s=5)
                        fig.colorbar(scatter, ax=ax, label='Frame index')
                        ax.set_xlabel('Phi (°)')
                        ax.set_ylabel('Psi (°)')
                        ax.set_xlim(-180, 180)
                        ax.set_ylim(-180, 180)
                        ax.set_title(f'Torsione #{i+1} - Mappa Phi vs Psi')
                        ax.grid(True)
                        plt.show()
                        
                        fig_cont["fig"] = fig
                    except Exception as e:
                        print(f"Errore nella selezione atomi o nel calcolo: {e}")
            
            btn_calc = widgets.Button(description=f"Calcola e Plotta torsione #{i+1}")
            btn_calc.on_click(calcola_e_plotta)
            
            def esporta_file(b, fig_cont=fig_container,
                             fname_widget=filename_widget,
                             fmt_widget=format_widget):
                if fig_cont["fig"] is None:
                    print("Nessun grafico da salvare, esegui prima il calcolo.")
                    return
                fname = fname_widget.value.strip()
                if fname == "":
                    print("Inserisci un nome file valido.")
                    return
                ext = fmt_widget.value
                full_filename = f"{fname}.{ext}"
                try:
                    fig_cont["fig"].savefig(full_filename)
                    print(f"Grafico salvato come {full_filename}")
                except Exception as e:
                    print(f"Errore nel salvataggio: {e}")
            
            btn_save = widgets.Button(description="Esporta grafico")
            btn_save.on_click(esporta_file)
            
            torsion_blocks.append(VBox([
                phi_block,
                psi_block,
                btn_calc,
                HBox([filename_widget, format_widget, btn_save]),
                output_plot
            ]))
        
        display(VBox(torsion_blocks))

num_torsions_widget.observe(genera_widgets_torsioni, names='value')
num_torsions_widget.disabled = True

display(num_torsions_widget, output_torsions)


BoundedIntText(value=1, description='N° torsioni:', disabled=True, max=10, min=1)

Output()